# Llama 3.2

The Meta Llama 3.2 collection of multilingual LLMs is a collection of pretrained and instruction tuned generative model in 8B. The Llama 3.1 instruction tuned text only models are optimised for multilingual dialogue use cases and outperform many of the available open source and closed chat models on common industry benchmarks.

Llama 3.2 is an auto-regressive language model that uses an optimised transformer architecture. The tuned versions are supervised fine-tuning (SFT) and reinforcement learning with human feedback (RLHF) to align with human preferences for helpfulness and safety.

### Transformer-Based Architecture
- Llama uses a decoder-only transformer architecture, which is similar in structure to models like GPT. In a decoder-only architecture, the model generates text by predicting the next token in a sequence, making it suitable for tasks like text generation, question answering and summarisation.

- Llama uses self-attention with multi-headed attention layers, which allows the model to capture relationships between words and understand context over varying distances.

### Pre-Normalization and RMSNorm
- **Layer Normalization:** Llama uses RMSNorm (Root Mean Square Normalization) instead of LayerNorm, which is the normalization technique typically used in transformers. RMSNorm normalizes each layer by the root mean square of its values rather than the mean and variance.

- **Pre-Normalization:** The model applies normalization layers before the attention and feed-forward layers instead of after. This change has shown to stabilise training and reduce issues that can arise during deep transformer training.

### Rotary Positional Embeddings (RoPE)
- **Positional Encoding:** Llama uses Rotary Positional Embeddings (RoPE), which were also used in models like GPT-NeoX and GPT-J. Unlike absolute positional encodings (used in BERT) or learned positional encodings, RoPE is a form of relative positional encoding that enhances the model's ability to capture position-dependent relationships between tokens.

- **Extended Context Length:** RoPE also allows for efficient scaling of the context window, meaning Llama can handle longer sequences more effectively than traditional transformers with fixed positional encodings. This is useful for tasks that require understanding longer documents or paragraphs.

### GeLU Activation
- Llama uses the GeLU (Gaussian Error Linear Unit) activation function, which is common in modern transformers. GeLU is smoother than ReLU and has shown to improve convergence and overall performance in deep learning models.


In [1]:
# %pip install  --upgrade "transformers==4.40.0" "datasets==2.18.0" "accelerate==0.29.3" "evaluate==0.4.1" "bitsandbytes==0.43.1" "huggingface_hub==0.22.2" "trl==0.8.6" "peft==0.10.0"

In [ ]:
# %pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('final_df.csv')

In [3]:
data.head()

,year,month,sentiment,processed_full_review
0,2024,3,Neutral,ok use airlin go singapor london heathrow issu...
1,2024,3,Negative,don give money book paid receiv email confirm ...
2,2024,3,Positive,best airlin world best airlin world seat food ...
3,2024,3,Negative,premium economi seat singapor airlin not worth...
4,2024,3,Negative,imposs get promis refund book flight full mont...


In [4]:
data['sentiment'].value_counts()

sentiment
Positive    7913
Negative    2441
Neutral     1164
Name: count, dtype: int64

In [5]:
data = data.sample(frac=1, random_state=85).reset_index(drop=True)

# Split the DataFrame
train_size = 0.8
eval_size = 0.1

# Calculate sizes
train_end = int(train_size * len(data))
eval_end = train_end + int(eval_size * len(data))

# Split the data
X_train = data[:train_end]
X_eval = data[train_end:eval_end]
X_test = data[eval_end:]

# Define the prompt generation functions
def generate_prompt(data_point):
    return f"""
            Classify the text into Positive, Neutral, Negative, and return the answer as the corresponding airline sentiment label.
text: {data_point["processed_full_review"]}
label: {data_point["sentiment"]}""".strip()

def generate_test_prompt(data_point):
    return f"""
            Classify the text into Positive, Neutral, Negative, and return the answer as the corresponding airline sentiment label.
text: {data_point["processed_full_review"]}
label: """.strip()

# Generate prompts for training and evaluation data
X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)

# Generate test prompts and extract true labels
y_true = X_test.loc[:,'sentiment']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

C:\Users\User\AppData\Local\Temp\ipykernel_17556\1204374432.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
C:\Users\User\AppData\Local\Temp\ipykernel_17556\1204374432.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)


In [8]:
X_train.sentiment.value_counts()

sentiment
Positive    6316
Negative    1975
Neutral      923
Name: count, dtype: int64

In [9]:
y_true.value_counts()

sentiment
Positive    803
Negative    234
Neutral     116
Name: count, dtype: int64

In [10]:
# Convert to datasets
train_data = Dataset.from_pandas(X_train[["text"]])
eval_data = Dataset.from_pandas(X_eval[["text"]])

In [11]:
train_data['text'][3]

'Classify the text into Positive, Neutral, Negative, and return the answer as the corresponding airline sentiment label.\ntext: great fligth flew jakarta transfer singapor san francisco flight jakarta hour late but staff provid everyon snack realli good chicken pasta flight hour layov singapor hour flight\nlabel: Positive'

In [12]:
# Initialize tokenizer and model
base_model_name = "meta-llama/Llama-3.2-1B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

In [13]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token_id = tokenizer.eos_token_id

In [14]:
def predict(test, model, tokenizer):
    y_pred = []
    categories = ["Positive", "Neutral", "Negative"]
    
    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["text"]
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens=2, 
                        temperature=0.1)
        
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("label:")[-1].strip()
        
        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("none")
    
    return y_pred

In [15]:
y_pred = predict(X_test, model, tokenizer)

100%|██████████| 1153/1153 [02:18<00:00,  8.31it/s]


In [16]:
def evaluate(y_true, y_pred):
    labels = ["Positive", "Neutral", "Negative"]
    mapping = {label: idx for idx, label in enumerate(labels)}
    
    def map_func(x):
        return mapping.get(x, -1)  # Map to -1 if not found, but should not occur with correct data
    
    y_true_mapped = np.vectorize(map_func)(y_true)
    y_pred_mapped = np.vectorize(map_func)(y_pred)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate accuracy report
    unique_labels = set(y_true_mapped)  # Get unique labels
    
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true_mapped)) if y_true_mapped[i] == label]
        label_y_true = [y_true_mapped[i] for i in label_indices]
        label_y_pred = [y_pred_mapped[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {labels[label]}: {label_accuracy:.3f}')
        
    # Generate classification report
    class_report = classification_report(y_true=y_true_mapped, y_pred=y_pred_mapped, target_names=labels, labels=list(range(len(labels))), digits=4)
    print('\nClassification Report:')
    print(class_report)
    
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true_mapped, y_pred=y_pred_mapped, labels=list(range(len(labels))))
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [17]:
evaluate(y_true, y_pred)

Accuracy: 0.739
Accuracy for label Positive: 0.903
Accuracy for label Neutral: 0.138
Accuracy for label Negative: 0.474

Classification Report:
              precision    recall  f1-score   support

    Positive     0.8065    0.9029    0.8519       803
     Neutral     0.1860    0.1379    0.1584       116
    Negative     0.6607    0.4744    0.5522       234

    accuracy                         0.7389      1153
   macro avg     0.5511    0.5051    0.5209      1153
weighted avg     0.7145    0.7389    0.7213      1153


Confusion Matrix:
[[725  51  27]
 [ 70  16  30]
 [104  19 111]]


In [ ]:
import bitsandbytes as bnb

# Find all linear layer names
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [19]:
modules = find_all_linear_names(model)
modules

['gate_proj', 'k_proj', 'v_proj', 'down_proj', 'q_proj', 'up_proj', 'o_proj']

In [ ]:
output_dir="llama-3.2-fine-tuned-model"

# Fine tune using QLoRA
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=1,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    logging_steps=1,                         
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=False,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="wandb",                  # report metrics to w&b
    eval_strategy="steps",              # save checkpoint every epoch
    eval_steps = 0.2
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=512,
    packing=False,
    dataset_kwargs={
    "add_special_tokens": False,
    "append_concat_token": False,
    }
)

Map: 100%|██████████| 1151/1151 [00:00<00:00, 14387.56 examples/s]
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\trl\trainer\sft_trainer.py:323: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\accelerate\accelerator.py:469: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [21]:
# Train model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: shaun-zhou-2022 (shaun-zhou-2022-singapore-management-university). Use `wandb login --relogin` to force relogin


  0%|          | 0/1151 [00:00<?, ?it/s]c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
  0%|          | 1/1151 [00:03<1:07:51,  3.54s/it]

{'loss': 6.2077, 'grad_norm': 0.810721218585968, 'learning_rate': 5.7142857142857145e-06, 'epoch': 0.0}


  0%|          | 2/1151 [00:06<1:01:10,  3.19s/it]

{'loss': 6.2041, 'grad_norm': 0.7819814085960388, 'learning_rate': 1.1428571428571429e-05, 'epoch': 0.0}


...

100%|██████████| 1151/1151 [1:07:49<00:00,  2.96s/it]

{'loss': 3.1121, 'grad_norm': 0.8045836687088013, 'learning_rate': 0.0, 'epoch': 1.0}


100%|██████████| 1151/1151 [1:08:01<00:00,  3.55s/it]

{'train_runtime': 4083.0623, 'train_samples_per_second': 2.257, 'train_steps_per_second': 0.282, 'train_loss': 3.2312729984237047, 'epoch': 1.0}


TrainOutput(global_step=1151, training_loss=3.2312729984237047, metrics={'train_runtime': 4083.0623, 'train_samples_per_second': 2.257, 'train_steps_per_second': 0.282, 'total_flos': 4989322401423360.0, 'train_loss': 3.2312729984237047, 'epoch': 0.9993488170175819})

In [23]:
import wandb
wandb.finish()
model.config.use_cache = True

eval/loss,█▄▂▁
eval/runtime,█▁▁▁
eval/samples_per_second,▁███
eval/steps_per_second,▁███
train/epoch,▁▁▁▁▁▁▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇███
train/grad_norm,▄▆█▄▄▃▃▄▄▄▆▅▅▆▃▂▁▃▃▂▂▃▃▂▂▁▁▂▅▂▂▂▄▃▃▄▃▂▄▂
train/learning_rate,█████▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁
train/loss,▇▆▆▆▅▃▄▁▄▄█▄▇▄▄▆▅▇▅▅▅▁▇▅▃▄▆▃▂▃▃▃▃▂▆▆▆▆▄▂
eval/loss,3.0007
eval/runtime,136.4706


In [24]:
# Save trained model and tokenizer
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('llama-3.2-fine-tuned-model\\tokenizer_config.json',
 'llama-3.2-fine-tuned-model\\special_tokens_map.json',
 'llama-3.2-fine-tuned-model\\tokenizer.json')

In [25]:
y_pred = predict(X_test, model, tokenizer)
evaluate(y_true, y_pred)

  0%|          | 0/1153 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 1153/1153 [17:34<00:00,  1.09it/s]

Accuracy: 0.874
Accuracy for label Positive: 0.940
Accuracy for label Neutral: 0.578
Accuracy for label Negative: 0.795

Classification Report:
              precision    recall  f1-score   support

    Positive     0.9618    0.9402    0.9509       803
     Neutral     0.4527    0.5776    0.5076       116
    Negative     0.8455    0.7949    0.8194       234

    accuracy                         0.8742      1153
   macro avg     0.7533    0.7709    0.7593      1153
weighted avg     0.8870    0.8742    0.8796      1153


Confusion Matrix:
[[755  41   7]
 [ 22  67  27]
 [  8  40 186]]
